In [147]:
import pandas as pd
from datetime import datetime

import plotly.graph_objects as go


In [148]:
df = pd.read_csv("./data/EURUSD=X.csv", parse_dates=True)

df['ema55'] = df['Close'].ewm(span=55).mean()
df['ema200'] = df['Close'].ewm(span=200).mean()

df['ema_diff'] = df['ema55'] - df['ema200']
df['crossover_type'] = df['ema_diff'].apply(
    lambda x: 'bullish' if x > 0 else 'bearish')

df = df.dropna()
df = df.sort_values(by='Date')
df = df.reset_index(drop=True)


In [149]:
fig = go.Figure(
    data=
        [go.Candlestick(x=df.index,  open=df['Open'], high=df['High'], low=df['Low'], close=df['Close']),
         go.Scatter(x=df.index, y=df['ema55'], name='ema55'),
         go.Scatter(x=df.index, y=df['ema200'], name='ema200')
        ])


fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()


In [150]:
start_cash = 1000
trades = pd.DataFrame(columns=[
    'entry_date', 'exit_date', 'entry_price', 'exit_price', 'crossover_type', 'pnL', 'target', 'sl', 'balance'
])

position = {
    'entry_date': None,
    'exit_date': None,
    'entry_price': None,
    'exit_price': None,
    'is_long': False,
    'is_short': False,
    'active': False,
    'crossover_type': None,
    'target': None,
    'sl': None,
    'size': None,
    'balance': None,
    'pnL': None
}


def clear_position():
    position['entry_date'] = None
    position['exit_date'] = None
    position['entry_price'] = None
    position['exit_price'] = None
    position['is_long'] = False
    position['is_short'] = False
    position['active'] = False
    position['start_crossover_type'] = None
    position['target'] = None
    position['sl'] = None


def open(row, size=1, sl=0, target=0):
    if row is None:
        raise Exception("Provide opening trade data")

    position['entry_date'] = row['Date']
    position['entry_price'] = row['Close']
    position['is_long'] = row['crossover_type'] == 'bullish'
    position['is_short'] = row['crossover_type'] == 'bearish'
    position['active'] = True
    position['crossover_type'] = row['crossover_type']
    position['target'] = target
    position['sl'] = sl
    position['size'] = size
    position['balance'] = start_cash

    cost = position['entry_price'] * position['size']
    balance_amount = start_cash - cost

    if balance_amount < 0:
        raise Exception("Not enough balance to open position")

    position['cost'] = cost
    position['balance'] = balance_amount


def close(row):
    if row is None:
        raise Exception("Provide closing trade data")

    if not position['active']:
        raise Exception("Cannot close an inactive position")

    position['exit_date'] = row['Date']
    position['exit_price'] = row['Close']
    position['pnL'] = (position['exit_price'] * position['size'] -
                       position['entry_price']) * position['size']

    if (position['is_short']):
        position['pnL'] = -position['pnL']

    position['balance'] = position['balance'] + position['pnL']

    global trades
    t_df = pd.DataFrame([position])
    trades = pd.concat([trades, t_df], ignore_index=True)
    clear_position()


df['position'] = None


In [153]:
start = 200

while start < len(df):

    print(df['crossover_type'][start],
          df['crossover_type'][start-1], start, len(df))

    if position['active']:
        # print(position, "Position")
        # print(candle, "Candle")

        candle = df.iloc[start]

        # diff = candle['Close'] - position['entry_price']
        candle_close = candle['Close'] * position['size']

        target_price = position['entry_price'] + \
            position['target'] * position['cost']

        stoploss = position['entry_price'] - \
            position['sl'] * position['cost']

        print(pd.Series(position))
        print(candle_close, target_price, stoploss)

        if position['is_long'] and (candle_close >= target_price or candle_close <= stoploss):
            close(candle)

        if position['is_short'] and (candle_close <= target_price or candle_close >= stoploss):
            close(candle)

         # Skip to next row if position is active
        start = start + 1
        continue

    if df['crossover_type'][start] != df['crossover_type'][start-1]:
        df['position'][start] = 'crossover'

        # print(df.iloc[start])

        open(df.iloc[start], target=0.04, sl=0.01, size=100)

    start = start + 1

print(start)


bearish bearish 200 1303
entry_date        2020-06-17
exit_date               None
entry_price          1.12657
exit_price              None
is_long                 True
is_short               False
active                  True
crossover_type       bullish
target                  0.04
sl                      0.01
size                     100
balance              887.343
pnL                     None
cost                 112.657
dtype: object
114.61710000000001 5.63285 0.0
bearish bearish 201 1303
bearish bearish 202 1303
bearish bearish 203 1303
bearish bearish 204 1303
bearish bearish 205 1303
bearish bearish 206 1303
bearish bearish 207 1303
bearish bearish 208 1303
bearish bearish 209 1303
bearish bearish 210 1303
bearish bearish 211 1303
bearish bearish 212 1303
bearish bearish 213 1303
bearish bearish 214 1303
bearish bearish 215 1303
bearish bearish 216 1303
bearish bearish 217 1303
bearish bearish 218 1303
bearish bearish 219 1303
bearish bearish 220 1303
bearish bearish 221 1303

/tmp/ipykernel_12509/3245663617.py:37: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_12509/2368694656.py:80: FutureWarning:

In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.

/tmp/ipykernel_12509/3245663617.py:37: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_12509/2368694656.py:80: FutureWarning:

In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.

/tmp/ipykernel_12509/3245663